Equal Weight SMP 500 Index fund


    every stock will have equal distrubution

In [ ]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from pprint import pprint

Importing smp 500 csv file

since this is using free resources, we have a year 2020 list of SMP500 stocks in a CSV file

We then save the CSV file into a pandas data frame
    

In [ ]:
stocks = pd.read_csv("sp_500_stocks.csv")

Aquire API token

IEX cloud API token (sandbox mode)

best practice is to make a secrets.py file and import the token from there... we would ignore this file in a git push

SINCE THIS IS A FREE SANDBOX API CODE, THERE IS NO REASON TO SAVE AS NEW FILE.... BUT KEEP IN MIND !!!!


In [ ]:
IEX_CLOUD_API_TOKEN = ''

# creating columns for pandas dataFrame  *** FOR LATER USER ***
my_columns = [  "Ticker"  , "Stock Price" , "Market Capitalization" , 'Numbers of Shares To Buy'   ]
final_dataframe = pd.DataFrame(  columns=my_columns   )

API CALL (single requests)

we will for loop through the data frame and capture following data points

market capitalization for each stock

price of each stock



** Adding our stocks data to pandas DataFrame ** ---> [ticker , stock price , market cap , numbers of shares to buy]

In [ ]:
# for symbol in stocks['Ticker']:
#     api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
#     data = requests.get(api_url).json()
#     price = data['latestPrice']
#     market_cap = data['marketCap']
#     # adding a row to the pandas dataFrame
#     # WHEN APPENDING TO PANDAS DATAFRAMES, WE NEED TO ADD THE  ignore_index=True as a second parameter
#     # PANDAS SERIES NEEDS INDEXING FROM my_colmns var as a second parameter
#     final_dataframe = final_dataframe.append(  pd.Series( [symbol , price , market_cap , "N/A"] , index=my_columns )    ,   ignore_index=True  )

# # final_dataframe

BATCH API CALLS!!!!!!!!!!

speeds up our code by calling for more data all at once instead 500 single requests


In [ ]:
# function to split bigger list into smaller lists
def chunks( lst , n  ):
    ''' yeild successive n-sized lists from lst'''
    for i in range( 0 , len(lst) , n ):
        yield lst[i: i + n]

# 2 dimentional list with each inner list containing 100 tickers
symbol_groups = list(chunks( stocks['Ticker'] , 100  ) )
symbol_strings = []

# batch api calls for every list in symbol_groups, appends to final_dataframe
for i in range( 0 , len(symbol_groups) , 1  ):
    #takes all content in symbol_groups[i], adds them to symbol_strings[] with a comma between each stock
    symbol_strings.append( ",".join(symbol_groups[i])  )
    # print(symbol_groups[i])


# loop through every group
# batch call the api for 100 stocks at a time
# loop through each symbol in the group
# append those 100 stock's data into our final_dataframe
for string_group in symbol_strings:
    batch_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={string_group}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_call_url).json()
    for symbol in string_group.split(","):
        # print(data[symbol]['quote'])
        price = data[symbol]['quote']['latestPrice']
        market_cap = data[symbol]['quote']['marketCap']
        # adding a row to the pandas dataFrame
        # WHEN APPENDING TO PANDAS DATAFRAMES, WE NEED TO ADD THE  ignore_index=True as a second parameter
        # PANDAS SERIES NEEDS INDEXING FROM my_colmns var as a second parameter
        final_dataframe = final_dataframe.append(  pd.Series( [symbol , price , market_cap , "N/A"] , index=my_columns )    ,   ignore_index=True  )




CALCULATING THE NUMBER OF SHARES TO BUY

In [ ]:
# validate user input for portfollio size '$$$'
def user_portfollio_size_input(portfollio_size):
    try:
        val = float(portfollio_size)
        print(f"Your have entered: ${val}\n")
        return True
    except ValueError:
        print("ERROR: Please type a numerical value")
        return False
portfollio_size = input("Please enter value of your portfollio: $")
while not user_portfollio_size_input(portfollio_size):
    portfollio_size = input("Please enter value of your portfollio: $")

PORTION OUT EQUAL AMOUNTS INTO EACH STOCK FROM SMP-500

In [ ]:
val = float(portfollio_size)
position_size = val / len(final_dataframe.index)
# print(position_size)
# print(len(final_dataframe.index))

for x in range(  0   , len(final_dataframe.index) , 1 ):
    final_dataframe.loc[x , "Numbers of Shares To Buy"] =  position_size /  final_dataframe.loc[x , "Stock Price"]

# output
final_dataframe


SAVE PANDAS DATAFRAME INTO CSV FILE

In [ ]:
# init writer objects
writer = pd.ExcelWriter('recommended_trades.xlsx' , engine='xlsxwriter')
final_dataframe.to_excel(writer , "Recommended Trades" , index = False)

# writer style init
font_color = "ffffff"
background_color = "#0a0a23"

# format style init
string_format = writer.book.add_format(
    {
        'font_color' : font_color,
        "bg_color" : background_color,
        "border" : 1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format' : "$0.00",
        'font_color' : font_color,
        "bg_color" : background_color,
        "border" : 1
    }
)
float_format = writer.book.add_format(
    {
        'num_format' : "0.00",
        'font_color' : font_color,
        "bg_color" : background_color,
        "border" : 1
    }
)


# applying formats to .xlsx file
column_formats = {
    'A' : ['Ticker' , string_format],
    'B' : ['Stock Price' , dollar_format],
    'C' : ['Market Capitalization' , dollar_format],
    'D' : ['Number of Shares To Buy' , float_format],
}
for col in  column_formats.keys():
    writer.sheets['Recommended Trades'].set_column( f"{col}:{col}" , 18 , column_formats[col][1] )
    writer.sheets['Recommended Trades'].write( f"{col}1" , column_formats[col][0] , column_formats[col][1] )

# SAVE .xlsx
writer.save()
# SAVE INPUTS
with open("user_inputs.txt" , "w") as f:
    f.write( f"Amount Specified: ${val}\n"  )
    f.write( f"Amount Invested Into Each Stock: ${position_size}"  )